In [1]:
library(ExperimentHub)
library(Matrix)
library(scater)
library(reticulate)

Loading required package: BiocGenerics


Attaching package: 'BiocGenerics'


The following objects are masked from 'package:stats':

    IQR, mad, sd, var, xtabs


The following objects are masked from 'package:base':

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min


Loading required package: AnnotationHub

Loading required package: BiocFileCache

Loading required package: dbplyr

Loading required package: SingleCellExperiment

Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: 'MatrixGenerics'


The following objects are masked from 'package:m

In [2]:
#### user specified

data_folder = "D:/analyze_Pearson_residuals/"
data_subfolder = "lupus"

data_path  <- paste0 ( data_folder, data_subfolder )

In [3]:
# output data
counts_pkl = "sce_counts_nz_GE_1.pkl"
cells_pkl = "sce_row_data_nz_GE_1.pkl"
genes_pkl = "sce_column_data_nz_GE_1.pkl"


# paths: output data
counts_dsn     =  paste0 ( data_path, '/', counts_pkl )
cell_data_dsn  =  paste0 ( data_path, '/', cells_pkl )
gene_data_dsn  =  paste0 ( data_path, '/', genes_pkl )

In [4]:
eh <- ExperimentHub()
query(eh, "Kang")

(sce <- eh[["EH2259"]])
dim(sce)

sce <- sce[rowSums(counts(sce) > 0) > 0, ]
dim(sce)

snapshotDate(): 2022-10-31



ExperimentHub with 3 records
# snapshotDate(): 2022-10-31
# $dataprovider: NCI_GDC, GEO
# $species: Homo sapiens
# $rdataclass: character, SingleCellExperiment, BSseq
# additional mcols(): taxonomyid, genome, description,
#   coordinate_1_based, maintainer, rdatadateadded, preparerclass, tags,
#   rdatapath, sourceurl, sourcetype 
# retrieve records with, e.g., 'object[["EH1661"]]' 

           title                                               
  EH1661 | Whole Genome Bisulfit Sequencing Data for 47 samples
  EH1662 | Whole Genome Bisulfit Sequencing Data for 47 samples
  EH2259 | Kang18_8vs8                                         

snapshotDate(): 2022-10-31

see ?muscData and browseVignettes('muscData') for documentation

loading from cache



class: SingleCellExperiment 
dim: 35635 29065 
metadata(0):
assays(1): counts
rownames(35635): MIR1302-10 FAM138A ... MT-ND6 MT-CYB
rowData names(2): ENSEMBL SYMBOL
colnames(29065): AAACATACAATGCC-1 AAACATACATTTCC-1 ... TTTGCATGGTTTGG-1
  TTTGCATGTCTTAC-1
colData names(5): ind stim cluster cell multiplets
reducedDimNames(1): TSNE
mainExpName: NULL
altExpNames(0):

[1] 35635 29065

[1] 18890 29065

In [5]:
qc <- perCellQCMetrics(sce)

ol <- isOutlier(metric = qc$detected, nmads = 2, log = TRUE)
sce <- sce[, !ol]
dim(sce)

sce <- sce[rowSums(counts(sce) > 0) >= 1, ]
dim(sce)

[1] 18890 26820

[1] 18501 26820

In [6]:
count_data =  counts( sce )
count_matrix = as.matrix( count_data )
print ( dim ( count_matrix ) )
print ( head (count_matrix [, c(1:5)] ) )

sparse_matrix <- as(count_matrix, "dgTMatrix")  

Warning message in asMethod(object):
"sparse->dense coercion: allocating vector of size 3.7 GiB"


[1] 18501 26820
              AAACATACAATGCC-1 AAACATACATTTCC-1 AAACATACCAGAAA-1
RP11-34P13.8                 0                0                0
AL627309.1                   0                0                0
AP006222.2                   0                0                0
RP11-206L10.2                0                0                0
RP11-206L10.9                0                0                0
FAM87B                       0                0                0
              AAACATACCAGCTA-1 AAACATACCATGCA-1
RP11-34P13.8                 0                0
AL627309.1                   0                0
AP006222.2                   0                0
RP11-206L10.2                0                0
RP11-206L10.9                0                0
FAM87B                       0                0


as(<matrix>, "dgTMatrix") is deprecated since Matrix 1.5-0; do as(as(as(., "dMatrix"), "generalMatrix"), "TsparseMatrix") instead



In [7]:
col_data = colData( sce )

barcode = col_data@rownames
cluster = col_data@listData$cell
df_cell_data = data.frame ( barcode, cluster )
print ( dim ( df_cell_data ) )
print ( head ( df_cell_data ) )

[1] 26820     2
           barcode         cluster
1 AAACATACAATGCC-1     CD4 T cells
2 AAACATACATTTCC-1 CD14+ Monocytes
3 AAACATACCAGAAA-1 CD14+ Monocytes
4 AAACATACCAGCTA-1 CD14+ Monocytes
5 AAACATACCATGCA-1     CD4 T cells
6 AAACATACCTCGCT-1 CD14+ Monocytes


In [8]:
row_data = rowData( sce )

ENSEMBL_ID = row_data@listData$ENSEMBL
gene_symbol = row_data@rownames
df_gene_data = data.frame ( ENSEMBL_ID, gene_symbol )
print ( dim ( df_gene_data ) )
print ( head ( df_gene_data ) )

[1] 18501     2
       ENSEMBL_ID   gene_symbol
1 ENSG00000239945  RP11-34P13.8
2 ENSG00000237683    AL627309.1
3 ENSG00000228463    AP006222.2
4 ENSG00000228327 RP11-206L10.2
5 ENSG00000237491 RP11-206L10.9
6 ENSG00000177757        FAM87B


In [9]:
py_save_object ( sparse_matrix, counts_dsn )
py_save_object ( df_cell_data, cell_data_dsn )
py_save_object  ( df_gene_data, gene_data_dsn )